In [1]:
!git clone https://github.com/Samsung-IT-Academy/stepik-dl-nlp.git && pip install -r stepik-dl-nlp/requirements.txt
import sys; sys.path.append('/content/stepik-dl-nlp')

Cloning into 'stepik-dl-nlp'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 216 (delta 10), reused 18 (delta 4), pack-reused 190
Receiving objects: 100% (216/216), 42.11 MiB | 18.20 MiB/s, done.
Resolving deltas: 100% (89/89), done.
     |████████████████████████████████| 51kB 4.8MB/s 
     |████████████████████████████████| 1.7MB 5.4MB/s 
     |████████████████████████████████| 307kB 46.5MB/s 
     |████████████████████████████████| 7.1MB 26.6MB/s 
  Created wheel for ufal.udpipe: filename=ufal.udpipe-1.2.0.3-cp36-cp36m-linux_x86_64.whl size=5625771 sha256=b77e03af8ae332e789af2397af94ef18ee699cd29d448217e7913a5723f6094f
  Stored in directory: /root/.cache/pip/wheels/0c/9d/db/6d3404c33da5b7adb6c6972853efb6a27649d3ba15f7e9bebb
Successfully built ufal.udpipe


In [0]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import accuracy_score

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import collections

import torch
from torch import nn
from torch.nn import functional as F

import dlnlputils
from dlnlputils.data import tokenize_text_simple_regex, tokenize_corpus, build_vocabulary, \
    vectorize_texts, SparseFeaturesDataset
from dlnlputils.pipeline import train_eval_loop, predict_with_model, init_random_seed

init_random_seed()

import random

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [19]:
text = ["Казнить нельзя, помиловать. Нельзя наказывать.", "Казнить, нельзя помиловать. Нельзя освободить.", "Нельзя не помиловать.", "Обязательно освободить."]

tokenized_list_of_docs = tokenize_corpus(text, min_token_size=1)

print(tokenized_list_of_docs)

word_id = build_vocabulary(tokenized_list_of_docs, max_doc_freq=1, min_count=1)
print(word_id)
print()
# уже отсортировано по алфавиту и по частоте
word_id = list(zip(list(word_id[0].keys()), list(word_id[1])))
print(word_id)
print()
word_id = sorted(word_id, key=lambda x: (x[1], x[0]))
print(word_id)
print()

print(' '.join(list(map(lambda x: x[0], word_id))))
print(' '.join(list(map(lambda x: str(x[1]), word_id))))
#"""print(' '.join(list(word_id[0].keys())))
#print(list(word_id[1]))
#a = list(word_id[1])
#a = list(map(lambda x: str(x), a))
#print(' '.join(a))"""


[['казнить', 'нельзя', 'помиловать', 'нельзя', 'наказывать'], ['казнить', 'нельзя', 'помиловать', 'нельзя', 'освободить'], ['нельзя', 'не', 'помиловать'], ['обязательно', 'освободить']]
({'помиловать': 0, 'нельзя': 1, 'казнить': 2, 'освободить': 3, 'наказывать': 4, 'не': 5, 'обязательно': 6}, array([0.75, 0.75, 0.5 , 0.5 , 0.25, 0.25, 0.25], dtype=float32))

[('помиловать', 0.75), ('нельзя', 0.75), ('казнить', 0.5), ('освободить', 0.5), ('наказывать', 0.25), ('не', 0.25), ('обязательно', 0.25)]

[('наказывать', 0.25), ('не', 0.25), ('обязательно', 0.25), ('казнить', 0.5), ('освободить', 0.5), ('нельзя', 0.75), ('помиловать', 0.75)]

наказывать не обязательно казнить освободить нельзя помиловать
0.25 0.25 0.25 0.5 0.5 0.75 0.75


In [0]:
import numpy as np
import scipy.sparse
import torch
from torch.utils.data import Dataset


def vectorize_texts(tokenized_texts, word2id, word2freq, mode='tfidf', scale=True):
    assert mode in {'tfidf', 'idf', 'tf', 'bin', 'ltfidf'}

    # считаем количество употреблений каждого слова в каждом документе
    result = scipy.sparse.dok_matrix((len(tokenized_texts), len(word2id)), dtype='float32')
    for text_i, text in enumerate(tokenized_texts):
        for token in text:
            if token in word2id:
                result[text_i, word2id[token]] += 1

    # получаем бинарные вектора "встречается или нет"
    if mode == 'bin':
        result = (result > 0).astype('float32')

    # получаем вектора относительных частот слова в документе
    elif mode == 'tf':
        result = result.tocsr()
        result = result.multiply(1 / result.sum(1))

    # полностью убираем информацию о количестве употреблений слова в данном документе,
    # но оставляем информацию о частотности слова в корпусе в целом
    elif mode == 'idf':
        result = (result > 0).astype('float32').multiply(1 / word2freq)

    # учитываем всю информацию, которая у нас есть:
    # частоту слова в документе и частоту слова в корпусе
    elif mode == 'tfidf':
        result = result.tocsr()
        result = result.multiply(1 / result.sum(1))  # разделить каждую строку на её длину
        result = result.multiply(1 / word2freq)  # разделить каждый столбец на вес слова

    elif mode == 'ltfidf':
        result = result.tocsr()
        result = result.multiply(1 / result.sum(1))  # разделить каждую строку на её длину
        result = scipy.sparse.csr_matrix.toarray(scipy.sparse.dok_matrix.tocsr(result))
        result = np.log(result + 1)
        result = scipy.sparse.csr_matrix(result)
        result = result.multiply(1 / word2freq)  # разделить каждый столбец на вес слова

    if scale:
        result = result.tocsc()
        #result -= result.min()
        #result /= (result.max() + 1e-6)

        result = scipy.sparse.csr_matrix.toarray(scipy.sparse.dok_matrix.tocsr(result))
        std = result.std(0, ddof=1)
        mean = result.mean(0)

        result = (result - mean) / std
        result = scipy.sparse.csr_matrix(result)



    return result.tocsr()

In [0]:
train_tokenized = tokenize_corpus(text, min_token_size=1)

In [0]:
def build_vocabulary2(tokenized_texts, max_size=1000000, max_doc_freq=0.8, min_count=5, pad_word=None):
    word_counts = collections.defaultdict(int)
    # количество документов, в которых встречается данное слово
    doc_n = 0

    # посчитать количество документов, в которых употребляется каждое слово
    # а также общее количество документов
    for txt in tokenized_texts:
        doc_n += 1
        unique_text_tokens = set(txt)
        for token in unique_text_tokens:
            word_counts[token] += 1

    # убрать слишком редкие и слишком частые слова
    word_counts = {word: cnt for word, cnt in word_counts.items()
                   if cnt >= min_count and cnt / doc_n <= max_doc_freq}

    # отсортировать слова по убыванию частоты
    sorted_word_counts = sorted(word_counts.items(),
                                key=lambda pair: (pair[1], pair[0]))

    # добавим несуществующее слово с индексом 0 для удобства пакетной обработки
    if pad_word is not None:
        sorted_word_counts = [(pad_word, 0)] + sorted_word_counts

    # если у нас по прежнему слишком много слов, оставить только max_size самых частотных
    if len(word_counts) > max_size:
        sorted_word_counts = sorted_word_counts[:max_size]

    # нумеруем слова
    word2id = {word: i for i, (word, _) in enumerate(sorted_word_counts)}

    # нормируем частоты слов
    word2freq = np.array([cnt / doc_n for _, cnt in sorted_word_counts], dtype='float32')

    return word2id, word2freq

In [0]:
vocabulary, word_doc_freq = build_vocabulary2(train_tokenized, max_doc_freq=1, min_count=1)

In [0]:
VECTORIZATION_MODE = 'ltfidf'
train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

In [24]:
train_vectors

<4x7 sparse matrix of type '<class 'numpy.float32'>'
	with 28 stored elements in Compressed Sparse Row format>

In [0]:
result = scipy.sparse.csr_matrix.toarray(scipy.sparse.dok_matrix.tocsr(train_vectors))

In [27]:
list(result)

[array([ 1.5000001 , -0.5       , -0.5       ,  0.8660254 , -0.76301265,
         0.59546685,  0.16096792], dtype=float32),
 array([-0.50000006, -0.5       , -0.5       ,  0.8660254 ,  0.18368238,
         0.59546685,  0.16096792], dtype=float32),
 array([-0.50000006,  1.5       , -0.5       , -0.8660254 , -0.76301265,
         0.2938242 ,  1.042435  ], dtype=float32),
 array([-0.50000006, -0.5       ,  1.5       , -0.8660254 ,  1.342343  ,
        -1.4847579 , -1.364371  ], dtype=float32)]

In [29]:
for doc in result:
    print(' '.join(list(map(str, list(doc)))))
    print()

1.5000001 -0.5 -0.5 0.8660254 -0.76301265 0.59546685 0.16096792

-0.50000006 -0.5 -0.5 0.8660254 0.18368238 0.59546685 0.16096792

-0.50000006 1.5 -0.5 -0.8660254 -0.76301265 0.2938242 1.042435

-0.50000006 -0.5 1.5 -0.8660254 1.342343 -1.4847579 -1.364371

